# EDA

## Feature Engineering Plan

### 1. Core Raw Inputs

* **OHLCV features**

  * `Open`, `High`, `Low`, `Close`, `Adj Close`, `Volume`
* **Price series used**

  * `Adj Close` (adjusted for splits and dividends)

---

### 2. Prediction Target

* **Next-day return (preferred: log return)**

  * `target_logret_1d = log(AdjClose_{t+1} / AdjClose_t)`

---

### 3. Endogenous Features (from the same stock)

#### 3.1 Returns & Momentum (Most Important)

**Daily returns**

* `logret_1d = log(AdjClose_t / AdjClose_{t-1})`
* `ret_1d = AdjClose_t / AdjClose_{t-1} - 1`

**Lagged returns (core time-series signal)**

* `logret_1d_lag_1` … `logret_1d_lag_60`
  *(60-step lookback sliding window)*

**Rolling momentum (mean of returns)**

* `ret_mean_5`
* `ret_mean_10`
* `ret_mean_20`
* `ret_mean_60`

> These capture short-, medium-, and long-term momentum persistence.

---

#### 3.2 Volatility (Critical for Financial Time Series)

**Rolling volatility (std. dev. of returns)**

* `ret_vol_5`
* `ret_vol_10`
* `ret_vol_20`
* `ret_vol_60`

**Range-based volatility proxies**

* `hl_range = (High - Low) / Close`
* `oc_change = (Close - Open) / Open`

---

#### 3.3 Trend Indicators (Computed on Adj Close)

**Simple Moving Averages (SMA)**

* `sma_5`, `sma_10`, `sma_20`, `sma_60`

**Trend strength (price relative to trend)**

* `sma_spread_5  = AdjClose / sma_5  - 1`
* `sma_spread_20 = AdjClose / sma_20 - 1`
* `sma_spread_60 = AdjClose / sma_60 - 1`

**Exponential Moving Averages (EMA)**

* `ema_12`
* `ema_26`

**MACD family**

* `macd        = ema_12 - ema_26`
* `macd_signal = EMA(macd, 9)`
* `macd_hist   = macd - macd_signal`

---

#### 3.4 Mean Reversion / Overbought–Oversold Signals

* **Relative Strength Index**

  * `rsi_14`

**Bollinger Bands (20-day, 2σ)**

* `bb_mid_20`
* `bb_upper_20`
* `bb_lower_20`
* `bb_width_20 = (bb_upper_20 - bb_lower_20) / bb_mid_20`

---

#### 3.5 Volume-Based Features (Market Participation)

* `vol_change = Volume_t / Volume_{t-1} - 1`
* `vol_z_20   = (Volume_t - mean(Volume_{t-20:t-1})) / std(Volume_{t-20:t-1})`
* `obv` (On-Balance Volume)

---

### 4. Cross-Stock (Peer) Exogenous Features

For each target stock **T**, include returns from the other three stocks as exogenous inputs.

Example:

* If `T = AAPL`, peers = `{MSFT, GOOG, AMZN}`

**Peer return lags**

* `peer_logret_lag_1` … `peer_logret_lag_20`
  *(for each peer ticker)*

---

### 5. Market, Sector & Volatility Exogenous Features

#### 5.1 Market & Sector Exposure

* **SPY (S&P 500 ETF)**

  * `spy_logret_lag_1` … `spy_logret_lag_20`
* **XLK (Technology Sector ETF)**

  * `xlk_logret_lag_1` … `xlk_logret_lag_20`
* **QQQ (Nasdaq-100 ETF)**

  * `qqq_logret_lag_1` … `qqq_logret_lag_20`

---

#### 5.2 Market Risk / Volatility Sentiment

* **VIX Index (`^VIX`)**

  * Daily log returns or percentage changes
  * Acts as a leading indicator of near-term market uncertainty

---

### 6. Calendar Effects

* `dow` — day of week (0 = Monday, …, 4 = Friday)
* `month` — month of year (1–12)
* `is_month_end` — indicator variable (0/1)

---

### 7. Lag & Window Design Summary

* **Endogenous return lags:** 1 … 60
* **Exogenous lags:** 1 … 20
* **Rolling windows:** 5, 10, 20, 60



In [1]:
#cell 1: imports + project paths
from pathlib import Path
import numpy as np
import pandas as pd


In [2]:
#cell 2: project root + folders
cwd = Path.cwd().resolve()
project_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "data").exists():
        project_root = p
        break
if project_root is None:
    raise RuntimeError("Project root not found: missing /data folder")

DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

RAW_PATH = RAW_DIR / "yfinance_ohlcv_long.parquet"
if not RAW_PATH.exists():
    raise FileNotFoundError(f"Missing file: {RAW_PATH}")

print("project_root:", project_root)
print("raw_path:", RAW_PATH)
print("processed_dir:", PROCESSED_DIR)


project_root: /media/workhorse/Windows/MyWorkPlace/Projects/StockPredictor
raw_path: /media/workhorse/Windows/MyWorkPlace/Projects/StockPredictor/data/raw/yfinance_ohlcv_long.parquet
processed_dir: /media/workhorse/Windows/MyWorkPlace/Projects/StockPredictor/data/processed


In [3]:
#cell 3: load raw long ohlcv
raw = pd.read_parquet(RAW_PATH)

#required columns check
required = ["date", "ticker", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
missing = [c for c in required if c not in raw.columns]
if missing:
    raise ValueError(f"Missing columns in raw: {missing}")

raw["date"] = pd.to_datetime(raw["date"]).dt.tz_localize(None)
raw = raw.sort_values(["ticker", "date"]).reset_index(drop=True)

print("raw shape:", raw.shape)
raw.head()


raw shape: (39456, 8)


,date,ticker,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,AAPL,19.779285,19.821428,19.343929,19.608213,16.612209,560518000.0
1,2013-01-03,AAPL,19.567142,19.631071,19.321428,19.360714,16.402523,352965200.0
2,2013-01-04,AAPL,19.177500,19.236786,18.779642,18.821428,15.945646,594333600.0
3,2013-01-07,AAPL,18.642857,18.903570,18.400000,18.710714,15.851843,484156400.0
4,2013-01-08,AAPL,18.900356,18.996071,18.616072,18.761070,15.894506,458707200.0


In [4]:
#cell 4: config
TARGET_TICKERS = ["AAPL", "MSFT", "GOOG", "AMZN"]

EXOG_TICKERS = [
    "SPY", "QQQ", "XLK",
    "^VIX", "^TNX",
    "UUP", "CL=F", "GC=F"
]

ALL_TICKERS = TARGET_TICKERS + EXOG_TICKERS

LAGS_ENDOG = list(range(1, 61))
LAGS_EXOG = list(range(1, 21))
ROLL_WINDOWS = [5, 10, 20, 60]

df = raw[raw["ticker"].isin(ALL_TICKERS)].copy()
df = df.sort_values(["ticker", "date"]).reset_index(drop=True)

print("kept tickers:", sorted(df["ticker"].unique().tolist()))
print("df shape:", df.shape)


kept tickers: ['AAPL', 'AMZN', 'CL=F', 'GC=F', 'GOOG', 'MSFT', 'QQQ', 'SPY', 'UUP', 'XLK', '^TNX', '^VIX']
df shape: (39456, 8)


In [5]:
#cell: remove rows where target tickers have missing Adj Close
mask_bad_target_px = df["ticker"].isin(TARGET_TICKERS) & df["Adj Close"].isna()
print("dropping rows (targets with missing Adj Close):", int(mask_bad_target_px.sum()))

df = df.loc[~mask_bad_target_px].copy()
df = df.sort_values(["ticker", "date"]).reset_index(drop=True)

print("df shape after drop:", df.shape)


dropping rows (targets with missing Adj Close): 12
df shape after drop: (39444, 8)


In [6]:
#cell 5: core returns + target + range proxies
df["adj_close"] = df["Adj Close"].astype(float)

g = df.groupby("ticker")["adj_close"]

#returns
df["logret_1d"] = g.transform(lambda s: np.log(s) - np.log(s.shift(1)))
df["ret_1d"] = g.pct_change()

#target
df["target_logret_1d"] = df.groupby("ticker")["logret_1d"].shift(-1)

#range proxies
df["hl_range"] = (df["High"] - df["Low"]) / df["Close"]
df["oc_change"] = (df["Close"] - df["Open"]) / df["Open"]


/media/workhorse/Windows/MyWorkPlace/ml_venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_25300/847140783.py:8: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["ret_1d"] = g.pct_change()


In [7]:
#cell 6: rolling momentum + rolling volatility on ret_1d
for w in ROLL_WINDOWS:
    df[f"ret_mean_{w}"] = df.groupby("ticker")["ret_1d"].transform(lambda s: s.rolling(w, min_periods=w).mean())
    df[f"ret_vol_{w}"]  = df.groupby("ticker")["ret_1d"].transform(lambda s: s.rolling(w, min_periods=w).std())


In [8]:
#cell 7: trend indicators (SMA, spreads, EMA, MACD) on Adj Close
for w in ROLL_WINDOWS:
    df[f"sma_{w}"] = df.groupby("ticker")["adj_close"].transform(lambda s: s.rolling(w, min_periods=w).mean())

df["sma_spread_5"]  = df["adj_close"] / df["sma_5"]  - 1
df["sma_spread_20"] = df["adj_close"] / df["sma_20"] - 1
df["sma_spread_60"] = df["adj_close"] / df["sma_60"] - 1

df["ema_12"] = df.groupby("ticker")["adj_close"].transform(lambda s: s.ewm(span=12, adjust=False).mean())
df["ema_26"] = df.groupby("ticker")["adj_close"].transform(lambda s: s.ewm(span=26, adjust=False).mean())

df["macd"] = df["ema_12"] - df["ema_26"]
df["macd_signal"] = df.groupby("ticker")["macd"].transform(lambda s: s.ewm(span=9, adjust=False).mean())
df["macd_hist"] = df["macd"] - df["macd_signal"]


In [9]:
#cell 8: RSI(14) on Adj Close
def rsi_14(series: pd.Series) -> pd.Series:
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.ewm(alpha=1/14, adjust=False, min_periods=14).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False, min_periods=14).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

df["rsi_14"] = df.groupby("ticker")["adj_close"].transform(rsi_14)


In [10]:
#cell 9: Bollinger Bands (20, 2sigma) on Adj Close
df["bb_mid_20"] = df.groupby("ticker")["adj_close"].transform(lambda s: s.rolling(20, min_periods=20).mean())
df["bb_std_20"] = df.groupby("ticker")["adj_close"].transform(lambda s: s.rolling(20, min_periods=20).std())

df["bb_upper_20"] = df["bb_mid_20"] + 2 * df["bb_std_20"]
df["bb_lower_20"] = df["bb_mid_20"] - 2 * df["bb_std_20"]
df["bb_width_20"] = (df["bb_upper_20"] - df["bb_lower_20"]) / df["bb_mid_20"]


In [11]:
#cell 10: volume features + OBV
df["vol_change"] = df.groupby("ticker")["Volume"].pct_change()

vol_mean_20 = df.groupby("ticker")["Volume"].transform(lambda s: s.rolling(20, min_periods=20).mean())
vol_std_20  = df.groupby("ticker")["Volume"].transform(lambda s: s.rolling(20, min_periods=20).std())
df["vol_z_20"] = (df["Volume"] - vol_mean_20) / vol_std_20

#OBV
price_diff = df.groupby("ticker")["adj_close"].diff()
direction = np.sign(price_diff).fillna(0.0)
df["obv"] = (direction * df["Volume"]).groupby(df["ticker"]).cumsum()


/tmp/ipykernel_25300/2735892375.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["vol_change"] = df.groupby("ticker")["Volume"].pct_change()


In [12]:
#cell 11: calendar features (based on date)
df["dow"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month
df["is_month_end"] = df["date"].dt.is_month_end.astype(int)


In [13]:
#cell 12: endogenous lags (logret_1d_lag1 ... logret_1d_lag60)
for L in LAGS_ENDOG:
    df[f"logret_1d_lag{L}"] = df.groupby("ticker")["logret_1d"].shift(L)


/tmp/ipykernel_25300/2467603668.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"logret_1d_lag{L}"] = df.groupby("ticker")["logret_1d"].shift(L)


In [14]:
#cell 13: build exogenous lag table (lag1 ... lag20) for all tickers, wide by date
exog_src = TARGET_TICKERS + EXOG_TICKERS

exog = df[df["ticker"].isin(exog_src)][["date", "ticker", "logret_1d"]].copy()
exog = exog.sort_values(["ticker", "date"]).reset_index(drop=True)

for L in LAGS_EXOG:
    exog[f"logret_lag{L}"] = exog.groupby("ticker")["logret_1d"].shift(L)

wide = exog.pivot(index="date", columns="ticker", values=[f"logret_lag{L}" for L in LAGS_EXOG])
wide.columns = [f"{t}_logret_lag{lag}" for lag, t in wide.columns]
wide = wide.reset_index()

print("exog wide shape:", wide.shape)
wide.head()


exog wide shape: (3288, 241)


,date,AAPL_logret_laglogret_lag1,AMZN_logret_laglogret_lag1,CL=F_logret_laglogret_lag1,GC=F_logret_laglogret_lag1,GOOG_logret_laglogret_lag1,MSFT_logret_laglogret_lag1,QQQ_logret_laglogret_lag1,SPY_logret_laglogret_lag1,UUP_logret_laglogret_lag1,...,CL=F_logret_laglogret_lag20,GC=F_logret_laglogret_lag20,GOOG_logret_laglogret_lag20,MSFT_logret_laglogret_lag20,QQQ_logret_laglogret_lag20,SPY_logret_laglogret_lag20,UUP_logret_laglogret_lag20,XLK_logret_laglogret_lag20,^TNX_logret_laglogret_lag20,^VIX_logret_laglogret_lag20
0,2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-04,-0.012703,0.004537,-0.002150,-0.008448,0.000581,-0.013487,-0.005222,-0.002262,0.007761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-07,-0.028249,0.002589,0.001828,-0.015414,0.019568,-0.018893,-0.003297,0.004382,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-08,-0.005900,0.035295,0.001074,-0.001579,-0.004373,-0.001871,0.000301,-0.002736,-0.003188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#cell 14: modeling panel for target tickers (merge exogenous + peer + market features)
#keeps all engineered features for the target ticker + adds lagged returns for exog/peers
target_df = df[df["ticker"].isin(TARGET_TICKERS)].copy()
panel = target_df.merge(wide, on="date", how="left")

#drop duplicated self-lags from wide (target already has logret_1d_lag1..60)
self_cols = []
for t in TARGET_TICKERS:
    for L in LAGS_EXOG:
        self_cols.append(f"{t}_logret_lag{L}")
panel = panel.drop(columns=[c for c in self_cols if c in panel.columns], errors="ignore")

print("panel shape:", panel.shape)
panel[["date", "ticker", "adj_close", "logret_1d", "target_logret_1d"]].head()


panel shape: (13140, 346)


,date,ticker,adj_close,logret_1d,target_logret_1d
0,2013-01-02,AAPL,16.612209,NaN,-0.012703
1,2013-01-03,AAPL,16.402523,-0.012703,-0.028249
2,2013-01-04,AAPL,15.945646,-0.028249,-0.005900
3,2013-01-07,AAPL,15.851843,-0.005900,0.002688
4,2013-01-08,AAPL,15.894506,0.002688,-0.015752


In [16]:
#cell 16: NA diagnostics (verify expected missing rows per ticker)
check_cols = [
    "logret_1d", "target_logret_1d",
    "ret_mean_60", "ret_vol_60",
    "logret_1d_lag60",
    "bb_mid_20", "rsi_14", "vol_z_20"
]

na_counts = (
    panel.groupby("ticker")[check_cols]
         .apply(lambda g: g.isna().sum())
         .reset_index()
)

print(na_counts)


  ticker  logret_1d  target_logret_1d  ret_mean_60  ret_vol_60  \
0   AAPL          1                 1           60          60   
1   AMZN          1                 1           60          60   
2   GOOG          1                 1           60          60   
3   MSFT          1                 1           60          60   

   logret_1d_lag60  bb_mid_20  rsi_14  vol_z_20  
0               61         19      14        19  
1               61         19      14        19  
2               61         19      14        19  
3               61         19      14        19  


In [17]:
#cell: diagnose missing prices that create extra return NAs
bad_px = df[df["ticker"].isin(TARGET_TICKERS) & df["Adj Close"].isna()][
    ["ticker", "date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
].copy()

print("missing Adj Close rows (targets):", bad_px.shape[0])
bad_px.sort_values(["ticker", "date"]).head(50)


missing Adj Close rows (targets): 0


,ticker,date,Open,High,Low,Close,Adj Close,Volume


In [18]:
#cell: show where logret_1d becomes NA (beyond the first row)
tmp = df[df["ticker"].isin(TARGET_TICKERS)][["ticker", "date", "Adj Close", "logret_1d"]].copy()
tmp["is_na_logret"] = tmp["logret_1d"].isna()

na_rows = tmp[tmp["is_na_logret"]].sort_values(["ticker", "date"])
print("NA logret_1d rows (targets):", na_rows.shape[0])
na_rows.head(50)


NA logret_1d rows (targets): 4


,ticker,date,Adj Close,logret_1d,is_na_logret
0,AAPL,2013-01-02,16.612209,NaN,True
3285,AMZN,2013-01-02,12.865500,NaN,True
13146,GOOG,2013-01-02,17.879629,NaN,True
16431,MSFT,2013-01-02,22.242884,NaN,True


In [19]:
#cell 15: save outputs
#1) full engineered long table for all tickers
out_long = PROCESSED_DIR / "engineered_long_all_tickers.parquet"
df.to_parquet(out_long, index=False)

#2) final modeling panel for the 4 target tickers (includes exogenous lagged returns)
out_panel = PROCESSED_DIR / "modeling_panel_targets.parquet"
panel.to_parquet(out_panel, index=False)

print("saved:", out_long)
print("saved:", out_panel)

saved: /media/workhorse/Windows/MyWorkPlace/Projects/StockPredictor/data/processed/engineered_long_all_tickers.parquet
saved: /media/workhorse/Windows/MyWorkPlace/Projects/StockPredictor/data/processed/modeling_panel_targets.parquet


In [22]:
#print all the columns in the final panel
print("final panel columns:")
for c in panel.columns:
    print(" -", c)
#panel shape
print("final panel shape:", panel.shape)

final panel columns:
 - date
 - ticker
 - Open
 - High
 - Low
 - Close
 - Adj Close
 - Volume
 - adj_close
 - logret_1d
 - ret_1d
 - target_logret_1d
 - hl_range
 - oc_change
 - ret_mean_5
 - ret_vol_5
 - ret_mean_10
 - ret_vol_10
 - ret_mean_20
 - ret_vol_20
 - ret_mean_60
 - ret_vol_60
 - sma_5
 - sma_10
 - sma_20
 - sma_60
 - sma_spread_5
 - sma_spread_20
 - sma_spread_60
 - ema_12
 - ema_26
 - macd
 - macd_signal
 - macd_hist
 - rsi_14
 - bb_mid_20
 - bb_std_20
 - bb_upper_20
 - bb_lower_20
 - bb_width_20
 - vol_change
 - vol_z_20
 - obv
 - dow
 - month
 - is_month_end
 - logret_1d_lag1
 - logret_1d_lag2
 - logret_1d_lag3
 - logret_1d_lag4
 - logret_1d_lag5
 - logret_1d_lag6
 - logret_1d_lag7
 - logret_1d_lag8
 - logret_1d_lag9
 - logret_1d_lag10
 - logret_1d_lag11
 - logret_1d_lag12
 - logret_1d_lag13
 - logret_1d_lag14
 - logret_1d_lag15
 - logret_1d_lag16
 - logret_1d_lag17
 - logret_1d_lag18
 - logret_1d_lag19
 - logret_1d_lag20
 - logret_1d_lag21
 - logret_1d_lag22
 - logret_1